In [1]:
import re
import polars as pl

from tqdm import tqdm

book_path = "lipu-sewi/bible/full.md"

part = ""
book = ""
chapter = ""
verse = 0

lipu = []

with open(book_path) as f:
    for line in tqdm(f.readlines()):
        line = line.strip()
        if not line:
            continue

        if line.startswith("## "):
            part, book = line.removeprefix("## ").split(": ")
            continue

        if line.startswith("### "):
            chapter = line.removeprefix("### " + book).strip()
            continue

        verse, verse_content = re.findall(r"(\d+): [\!\?] (.+)", line)[0]
        lipu.append({
            "part": part,
            "book": book,
            "chapter": int(chapter),
            "verse": int(verse),
            "tok": verse_content
        })

lipu_sewi = pl.DataFrame(lipu)
lipu_sewi

100%|██████████| 3904/3904 [00:00<00:00, 1997872.48it/s]


part,book,chapter,verse,tok
str,str,i64,i64,str
"""old testament""","""genesis""",1,1,"""open la, sewi Elowin li lon e …"
"""old testament""","""genesis""",1,2,"""ma li nasa li ala. pimeja li l…"
"""old testament""","""genesis""",1,3,"""sewi Elowin li toki. suno o lo…"
"""old testament""","""genesis""",1,4,"""sewi Elowin li lukin e suno: o…"
"""old testament""","""genesis""",1,5,"""sewi Elowin li nimi Tenpo Suno…"
…,…,…,…,…
"""new testament""","""revelation""",2,24,"""ona li awen pi ma Tasila. ona …"
"""new testament""","""revelation""",21,4,"""sewi Jawe li weka e telo ale t…"
"""apocrypha""","""tobit""",1,1,"""lipu ni li jo e toki pi jan To…"


In [2]:
lipu_sewi = lipu_sewi.with_columns(
    (lipu_sewi["book"] + "_" + lipu_sewi["chapter"] + "_" + lipu_sewi["verse"]).alias("key")
)

In [3]:
lipu_sewi.write_parquet("lipu-sewi.pq")

In [4]:
len(lipu_sewi["chapter"].unique())

37